##### 오차 역전파의 개념

오차 역전파(back propagation)
1. 임의의 초기 가중치(W)를 준 뒤 결과(y)를 계산한다
2. 계산 결과와 우리가 원하는 값 사이의 오차를 구한다
3. 경사 하강법을 이용해 바로 앞 가중치를 오차가 작아지는 방향으로 업데이트한다.
4. 위 과정을 더이상 오차가 줄어들지 않을 때까지 반복한다.

##### 코딩으로 확인하는 오차 역전파

1. __환경 변수 지정__ : 환경 변수에는 입력 값과 타깃 결과값이 포함된 데이터셋, 학습률 등이 포함됩니다. 또한, 활성화 함수와 가중치 등도 선언되어야 합니다.
2. __신경망 실행__ : 초깃값을 입력하여 활성화 함수와 가중치를 거쳐 결과값이 나오게 합니다.
3. __결과를 실제 값과 비교__ : 오차를 측정합니다.
4. __역전파 실행__ : 출력층와 은닉층의 가중치를 수정합니다.
5. __결과 출력__

In [7]:
# -*- coding: utf-8 -*-

import random
import numpy as np

random.seed(3)

# 환경 변수 지정

# 입력값 및 타겟값
data = [
    [[0, 0], [0]],
    [[0, 1], [1]],
    [[1, 0], [1]],
    [[1, 1], [0]]
]

# 실행 횟수(iterations), 학습률(lr), 모멘텀 계수(mo) 설정
iterations = 5000
lr = 0.1
mo = 0.4

# 활성화 함수 -1, 시그모이드
# 미분할 때와 아닐 때의 각각의 값
def sigmoid(x, derivative = False):
    if (derivative == True):
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))

# 활성화 함수 - 2, tanh
# tanh 함수의 미분은 1 - (활성화 함수 출력의 제곱)
def tanh(x, derivative = False):
    if (derivative == True):
        return 1 - x ** 2
    return np.tanh(x)

# 가중치 배열 만드는 함수
def makeMatrix(i, j, fill = 0.0):
    mat = []
    for i in range(i):
        mat.append([fill] * j)
    return mat

# 신경망의 실행
class NeuralNetwort:

    # 초깃값의 지정
    def __init__(self, num_x, num_yh, num_yo, bias = 1):

        # 입력값(num_x), 은닉층 초깃값(num_yh), 출력층 초깃값(num_yo), 바이어스
        self.num_x = num_x + bias   # bias는 1로 지정
        self.num_yh = num_yh
        self.num_yo = num_yo

        # 활성화 함수 초깃값
        self.activation_input = [1.0] * self.num_x
        self.activation_hidden = [1.0] * self.num_yh
        self.activation_out = [1.0] * self.num_yo

        # 가중치 입력 초깃값
        self.weight_in = makeMatrix(self.num_x, self.num_yh)
        for i in range(self.num_x):
            for j in range(self.num_yh):
                self.weight_in[i][j] = random.random()

        # 가중치 출력 초깃값
        self.weight_out = makeMatrix(self.num_yh, self.num_yo)
        for j in range(self.num_yh):
            for k in range(self.num_yo):
                self.weight_out[j][k] = random.random()

        # 모멘텀 SGD를 위한 이전 가중치 초깃값
        self.gradient_in = makeMatrix(self.num_x, self.num_yh)
        self.gradient_out = makeMatrix(self.num_yh, self.num_yo)

    # 업데이트 함수
    def update(self, inputs):

        # 입력 레이어의 활성화 함수
        for i in range(self.num_x - 1):
            self.activation_input[i] = inputs[i]

        # 은닉층의 활성화 함수
        for j in range(self.num_yh):
            sum = 0.0
            for i in range(self.num_x):
                sum = sum + self.activation_input[i] * self.weight_in[i][j]
            # 시그모이드와 tanh 중에서 활성화 함수 선택
            self.activation_hidden[j] = tanh(sum, False)

        # 출력층의 활성화 함수
        for k in range(self.num_yo):
            sum = 0.0
            for j in range(self.num_yh):
                sum = sum + self.activation_hidden[j] * self.weight_out[j][k]
            # 시그모이드와 tanh 중에서 활성화 함수 선택
            self.activation_out[k] = tanh(sum, False)

        return self.activation_out[:]

    # 역전파의 실행
    def backPropagate(self, targets):

        # 델타 출력 계산
        output_deltas = [0.0] * self.num_yo
        for k in range(self.num_yo):
            error = targets[k] - self.activation_out[k]
            # 시그모이드와 tanh 중에서 활성화 함수 선택, 미분 적용
            output_deltas[k] = tanh(self.activation_out[k], True) * error

        # 은닉 노드의 오차 함수
        hidden_deltas = [0.0] * self.num_yh
        for j in range(self.num_yh):
            error = 0.0
            for k in range(self.num_yo):
                error = error + output_deltas[k] * self.weight_out[j][k]
                # 시그모이드와 tanh 중에서 활성화 함수 선택, 미분 적용
            hidden_deltas[j] = tanh(self.activation_hidden[j], True) * error

        # 출력 가중치 업데이트
        for j in range(self.num_yh):
            for k in range(self.num_yo):
                gradient = output_deltas[k] * self.activation_hidden[j]
                v = mo * self.gradient_out[j][k] - lr * gradient
                self.weight_out[j][k] += v
                self.gradient_out[j][k] = gradient

        # 입력 가중치 업데이트
        for i in range(self.num_x):
            for j in range(self.num_yh):
                gradient = hidden_deltas[j] * self.activation_input[i]
                v = mo * self.gradient_in[i][j] - lr * gradient
                self.weight_in[i][j] += v
                self.gradient_in[i][j] = gradient

        # 오차의 계산(최소 제곱법)
        error = 0.0
        for k in range(len(targets)):
            error = error + 0.5 * (targets[k] - self.activation_out[k]) ** 2
        return error
        
    # 학습 실행
    def train(self, patterns):
        for i in range(iterations):
            error = 0.0
            for p in patterns:
                inputs = p[0]
                targets = p[1]
                self.update(inputs)
                error = error + self.backPropagate(targets)
            
            if i % 500 == 0:
                print(f'epochs : {i}, error : {error : .5f}')
    
    # 결과값 출력
    def result(self, patterns):
        for p in patterns:
            print(f'Input : {p[0]}, Predict : {self.update(p[0])}')

    
if __name__ == "__main__":

    # 두 개의 입력 값, 두 개의 레이어, 하나의 출력 값을 갖도록 설정
    n = NeuralNetwort(2, 2, 1)

    # 학습 실행
    n.train(data)

    # 결과값 출력
    n.result(data)


epochs : 0, error :  0.55964
epochs : 500, error :  0.00224
epochs : 1000, error :  0.00082
epochs : 1500, error :  0.00049
epochs : 2000, error :  0.00035
epochs : 2500, error :  0.00027
epochs : 3000, error :  0.00022
epochs : 3500, error :  0.00018
epochs : 4000, error :  0.00016
epochs : 4500, error :  0.00014
Input : [0, 0], Predict : [0.0005598180823600701]
Input : [0, 1], Predict : [0.9890208369438622]
Input : [1, 0], Predict : [0.9890523830082936]
Input : [1, 1], Predict : [0.0021901828422596003]
